# Sample mean and variance

Mathematically, the sample mean and the unbiased sample variance can be computed as follows

$$\bar{x}_n = \frac{1}{N}\sum_{i=1}^{n} x_i$$

The sum of the squared differences for the N samples
$$s_n^{2} = \frac{1}{N-1}\bigg( \sum_{i=1}^{n}\bigg(x_{i} - \bar{x}_{n}\bigg) ^ 2\bigg) $$

# Running statistics

How do we calculate these summaries from an infinite stream of values ?

Let's say we know the running mean $\bar{x}_{n-1}$ and variance $s_{n-1}^2$ and a new data sample $x_n$ has arrived.

1. What is the $\Delta_m$ (read Delta M) that should be added to old sample mean $\bar{x}_{n-1}$ to get the new sample mean $\bar{x}_{n}$ ?

2. What is the $\Delta_v$ that should be added to old sample variance $s_{n-1}^2$ to get the new sample variance $s_{n}^2$ ?


 


# $\Delta_m$ is straightforward

Let $N$ be the total number of samples after the sample $x_n$ has arrived. We have the following:

$$\bar{x}_n = \frac{(\bar{x}_{n-1})(N-1) + x_n}{N}$$


which gives us

$$\bar{x}_n = \bar{x}_{n-1} + \frac{x_n - \bar{x}_{n-1}}{N}$$

$$\bar{x}_n - \bar{x}_{n-1} = \frac{x_n - \bar{x}_{n-1}}{N}$$


$$\Delta_m = \frac{x_n - \bar{x}_{n-1}}{N}$$






# $\Delta_{v}$ is direct too

We will use Welford's algorithm to derive the difference. 

## Proof:

The difference between the sum of the squared differences of $N$ and $N-1$ samples can be written as follows


$$\Delta_{v} = (N-1)s_n^{2} - (N-2)s_{n-1}^{2}$$

$$\Delta_{v} = \sum_{i=1}^{n}\bigg(x_{i} - \bar{x}_{n}\bigg) ^ 2 - \sum_{i=1}^{n-1}\bigg(x_{i} - \bar{x}_{n-1}\bigg) ^ 2$$


$$\Delta_{v} =  \bigg(x_{i} - \bar{x}_{n}\bigg) ^ 2  + \sum_{i=1}^{n-1}\bigg(\bigg(x_{i} - \bar{x}_{n}\bigg) ^ 2 -\bigg(x_{i} - \bar{x}_{n-1}\bigg) ^ 2\bigg)$$


$$\Delta_{v} = (x_{n} - \bar{x}_n)(x_{n} - \bar{x}_{n-1})$$











# Code

In [17]:
import math
from typing import Union

class RunningStatistics:
    
    """
    Get running statistics for a infinite stream of values.    
    """
    
    def __init__(self):
        self._n = 0
        
        self._old_mean = 0.0
        self._old_variance = 0.0
        
        self._mean = 0.0
        self._variance = 0.0
        self._min, self._max = float("inf"), float("-inf") 
    
    def push(self, item: Union[int, float]):
        """
        Description: 
            Updates the statistics summary based on the new item. 
            The updation of all statistics but variance is trivial.
            To update variance, we use Welford's algorithm which is
            explained clearly at this link.
            https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance
        
        Args:
            item (int or float): Sample for which the statistics summary
            is to be updated.
        """
        self._n += 1
        
        if self._n == 1:
            self._mean = item
            self._min = item
            self._max = item
        else:
            self._min = min(self._min, item)
            self._max = max(self._max, item)
            self._mean = self._old_mean + (item - self._old_mean)/self._n
            self._variance = self._old_variance + (item - self._mean)*(item - self._old_mean)
        
        self._old_mean = self._mean
        self._old_variance = self._variance
    
    def __len__(self):
        return self._n
    
    @property
    def min(self):
        return self._min
    
    @property
    def max(self):
        return self._max
    
    @property
    def mean(self):
        return self._mean if self._n > 0 else 0.0
    
    @property
    def variance(self):
        return self._variance/(self._n-1) if self._n > 1 else 0.0
    
    @property
    def sd(self):
        return math.sqrt(self.variance)
    
    
    def get_state(self):
        """
        Description:
            Returns the state of the statistics summary so far calculated
            based on the items pushed.
        
        """
        return {
         "count": len(self),
         "min": self.min,
         "max": self.max,
         "mean": self.mean,
         "variance": self.variance,
         "standard_deviation": self.sd}

In [18]:
from pprint import pprint

stats = RunningStatistics()

data = [3, 4, 6, 8, 9, 10, 434, 45, 76, 13]

for item in data:
    stats.push(item)
    state = stats.get_state()
    pprint(state)

{'count': 1,
 'max': 3,
 'mean': 3,
 'min': 3,
 'standard_deviation': 0.0,
 'variance': 0.0}
{'count': 2,
 'max': 4,
 'mean': 3.5,
 'min': 3,
 'standard_deviation': 0.7071067811865476,
 'variance': 0.5}
{'count': 3,
 'max': 6,
 'mean': 4.333333333333333,
 'min': 3,
 'standard_deviation': 1.5275252316519468,
 'variance': 2.333333333333334}
{'count': 4,
 'max': 8,
 'mean': 5.25,
 'min': 3,
 'standard_deviation': 2.217355782608345,
 'variance': 4.916666666666667}
{'count': 5,
 'max': 9,
 'mean': 6.0,
 'min': 3,
 'standard_deviation': 2.5495097567963922,
 'variance': 6.5}
{'count': 6,
 'max': 10,
 'mean': 6.666666666666667,
 'min': 3,
 'standard_deviation': 2.804757862395017,
 'variance': 7.866666666666665}
{'count': 7,
 'max': 434,
 'mean': 67.71428571428571,
 'min': 3,
 'standard_deviation': 161.5371105821758,
 'variance': 26094.238095238095}
{'count': 8,
 'max': 434,
 'mean': 64.875,
 'min': 3,
 'standard_deviation': 149.76976378046785,
 'variance': 22430.98214285714}
{'count': 9,
 'max